In [45]:
from pathlib import Path
import dis
import marshal
import numpy as np
import types

with open("py_file.pyc", "r+b") as f:
    content = f.read(16)
    code_obj = marshal.load(f)
file_inside = dis.dis(code_obj)
bytecode = bytearray(code_obj.co_code)
bytecode = np.frombuffer(code_obj.co_code, dtype=np.uint8)

BINARY_ADD = dis.opmap["BINARY_ADD"]
BINARY_SUB = dis.opmap["BINARY_SUBTRACT"]

bytecode_new = np.where(bytecode == BINARY_ADD, BINARY_SUB, bytecode).astype(np.uint8)

new_code = types.CodeType(
    code_obj.co_argcount,
    code_obj.co_posonlyargcount if hasattr(code_obj, "co_posonlyargcount") else 0,
    code_obj.co_kwonlyargcount,
    code_obj.co_nlocals,
    code_obj.co_stacksize,
    code_obj.co_flags,
    bytecode_new.tobytes(),  
    code_obj.co_consts,
    code_obj.co_names,
    code_obj.co_varnames,
    code_obj.co_filename,
    code_obj.co_name,
    code_obj.co_firstlineno,
    code_obj.co_lnotab,
    code_obj.co_freevars,
    code_obj.co_cellvars
)

# Disassemble to see result
dis.dis(new_code)

  1           0 LOAD_CONST               0 (5)
              2 STORE_NAME               0 (x)

  2           4 LOAD_NAME                0 (x)
              6 LOAD_NAME                0 (x)
              8 BINARY_ADD
             10 STORE_NAME               1 (y)

  3          12 LOAD_NAME                0 (x)
             14 LOAD_NAME                0 (x)
             16 BINARY_SUBTRACT
             18 STORE_NAME               2 (z)

  4          20 LOAD_NAME                3 (print)
             22 LOAD_CONST               1 ('Hello World!')
             24 CALL_FUNCTION            1
             26 POP_TOP
             28 LOAD_CONST               2 (None)
             30 RETURN_VALUE
  1           0 LOAD_CONST               0 (5)
              2 STORE_NAME               0 (x)

  2           4 LOAD_NAME                0 (x)
              6 LOAD_NAME                0 (x)
              8 BINARY_SUBTRACT
             10 STORE_NAME               1 (y)

  3          12 LOAD_NAME           

In [48]:
with open("py_file.pyc", "wb") as f:
    f.write(content)                   # Keep original header
    marshal.dump(new_code, f)    # Write new code object


In [ ]:
import py_compile
py_compile.compile('py_file.py', cfile='py_file.pyc')

'py_file.pyc'

In [49]:
with open("py_file.pyc", "r+b") as f:
    content = f.read(16)
    code_obj_changed = marshal.load(f)
dis.dis(code_obj_changed)

  1           0 LOAD_CONST               0 (5)
              2 STORE_NAME               0 (x)

  2           4 LOAD_NAME                0 (x)
              6 LOAD_NAME                0 (x)
              8 BINARY_SUBTRACT
             10 STORE_NAME               1 (y)

  3          12 LOAD_NAME                0 (x)
             14 LOAD_NAME                0 (x)
             16 BINARY_SUBTRACT
             18 STORE_NAME               2 (z)

  4          20 LOAD_NAME                3 (print)
             22 LOAD_CONST               1 ('Hello World!')
             24 CALL_FUNCTION            1
             26 POP_TOP
             28 LOAD_CONST               2 (None)
             30 RETURN_VALUE


In [50]:
import sys
import tracemalloc

# Обычный класс без __slots__
class User:
    def __init__(self, name, surname, age, DOB, email_address, password_hash):
        self.name = name
        self.surname = surname
        self.age = age
        self.DOB = DOB
        self.email_address = email_address
        self.password_hash = password_hash

# Класс с __slots__
class SlottedUser:
    __slots__ = ['name', 'surname', 'age', 'DOB', 'email_address', 'password_hash']
    
    def __init__(self, name, surname, age, DOB, email_address, password_hash):
        self.name = name
        self.surname = surname
        self.age = age
        self.DOB = DOB
        self.email_address = email_address
        self.password_hash = password_hash

def measure_memory(Class, label):
    tracemalloc.start()

    users = [Class("John", "Doe", 30, "1990-01-01", "john@example.com", "hash123") for _ in range(10**6)]

    current, peak = tracemalloc.get_traced_memory()
    print(f"{label}:\n  Current memory usage: {current / 1024 / 1024:.2f} MB\n  Peak memory usage: {peak / 1024 / 1024:.2f} MB\n")

    # Clean up
    del users
    tracemalloc.stop()
   

In [51]:

measure_memory(User, "Without __slots__")
measure_memory(SlottedUser, "With __slots__")

Without __slots__:
  Current memory usage: 191.16 MB
  Peak memory usage: 191.16 MB

With __slots__:
  Current memory usage: 84.36 MB
  Peak memory usage: 84.36 MB

